# Physics Informed Neural Networks <br> F1 Car Front Wing Aerodymanics

## PINN

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import csv
from pinn import PINN
import utils

In [2]:
in_filepath = "/Users/ggito/repos/pinns/data/"
points_filename = "front_wing_points_final.csv"
norms_filename = "front_wing_norms_final.csv"

wing_df = pd.read_csv(in_filepath + points_filename)
norm_df = pd.read_csv(in_filepath + norms_filename)

print(wing_df)
print(norm_df)

              x         y         z
0      0.440148  0.373950  0.203123
1      0.713695  0.509429  0.155195
2      0.451790  0.321569  0.152462
3      0.032607  0.454912  0.208069
4      0.750952  0.439930  0.213273
...         ...       ...       ...
19995  0.913177  0.582509  0.151195
19996  0.115440  0.521203  0.128832
19997  0.453917  0.334118  0.152462
19998  0.556022  0.363779  0.201915
19999  0.030382  0.430567  0.111402

[20000 rows x 3 columns]
              x         y         z
0     -1.000000 -0.000293 -0.000429
1     -0.283259 -0.649111 -0.705988
2      0.000000  0.000000 -1.000000
3     -0.974604  0.223384  0.015692
4     -0.325352 -0.933390  0.151428
...         ...       ...       ...
19995 -0.018169 -0.132277  0.991046
19996 -0.991501 -0.031098 -0.126326
19997  0.000000  0.000000 -1.000000
19998  1.000000 -0.000293 -0.000429
19999 -0.730616  0.157085 -0.664473

[20000 rows x 3 columns]


In [3]:
x_max = 1
y_max = 1
z_max = 1
t_max = 1

Nx = 10
Ny = 10
Nz = 10
Nt = 10

dx = x_max / (Nx - 1)
dy = y_max / (Ny - 1)
dz = z_max / (Nz - 1)
dt = t_max / (Nt - 1)

x_test = np.linspace(0, x_max, Nx)
y_test = np.linspace(0, y_max, Ny)
z_test = np.linspace(0, z_max, Nz)
t_test = np.linspace(0, t_max, Nt)

x_grid, y_grid, z_gripd, t_grid = np.meshgrid(x_test, y_test, z_test, t_test)

In [4]:
input_dim = 4
output_dim = 4
# hidden_units = [32, 32, 32]
# hidden_units = [64, 64, 64, 64]
# hidden_units = [128, 128, 128, 128]
# hidden_units = [256, 256, 256, 256]
# hidden_units = [512, 512]
# hidden_units = [1024, 1024, 1024]
hidden_units = [1024, 1024, 1024, 1024]
# hidden_units = [2048, 2048, 2048, 2048]
# hidden_units = [20, 40, 80, 100, 100, 80, 40, 20]

if torch.backends.mps.is_available():
  device = torch.device("mps")
  x = torch.ones(1, device=device)
  print(x)
else:
  print("MPS device not found.")
  device = "cpu"

# device = "cpu"

pinn = PINN(input_dim, output_dim, hidden_units).to(device)

tensor([1.], device='mps:0')


In [5]:
# optimizer = torch.optim.Adam(pinn.parameters(), lr=0.001)
optimizer = torch.optim.LBFGS(pinn.parameters())

epochs = 305
Nf = 20000   # num of collocation points -> pde evaluation
N0 = 20000   # num of points to evaluate initial conditons
Nb = 20000   # num of points to evaluate boundary conditions
Nw = 20000   # num of points of the surface of the front wing to evaluate boundary conditions

# Density (rho): 1.2041kg/m^3
# Dynamic viscosity (mu): 1.81e-5 kg/m.s
rho = 1.2
mu = 1.81e-5

# m/s
in_velocity = 20

log_filepath = "/Users/ggito/repos/pinns/data/log.csv"

In [6]:
def sample_points_in_domain(min, max, num_of_samples):
  return np.random.uniform(min, max, size=num_of_samples)

In [7]:
def zeros(num):
  return np.zeros(num)

In [8]:
def ones(num):
  return np.ones(num)

In [9]:
def create_training_inputs(x_max, y_max, z_max, t_max, Nf, N0, Nb, Nw):
  # TODO: use quasi monte carlo sampling
  # collocation points
  x_f = utils.tensor_from_array(sample_points_in_domain(0, x_max, Nf), device=device, requires_grad=True)
  y_f = utils.tensor_from_array(sample_points_in_domain(0, y_max, Nf), device=device, requires_grad=True)
  z_f = utils.tensor_from_array(sample_points_in_domain(0, z_max, Nf), device=device, requires_grad=True)
  t_f = utils.tensor_from_array(sample_points_in_domain(0, t_max, Nf), device=device, requires_grad=True)
  # xyzt_f = utils.stack_xyzt_tensors(x_f, y_f, z_f, t_f)

  # initial condition points (t=0)
  x0 = utils.tensor_from_array(sample_points_in_domain(0, x_max, N0), device=device, requires_grad=False)
  y0 = utils.tensor_from_array(sample_points_in_domain(0, y_max, N0), device=device, requires_grad=False)
  z0 = utils.tensor_from_array(sample_points_in_domain(0, z_max, N0), device=device, requires_grad=False)
  t0 = utils.tensor_from_array(zeros(N0), device=device, requires_grad=False)
  xyzt_0 = utils.stack_xyzt_tensors(x0, y0, z0, t0)

  # boundary condition points (inflow, y=1)
  x_b = utils.tensor_from_array(sample_points_in_domain(0, x_max, Nb), device=device, requires_grad=False)
  y_b = utils.tensor_from_array(ones(Nb), device=device, requires_grad=False)
  z_b = utils.tensor_from_array(sample_points_in_domain(0, z_max, Nb), device=device, requires_grad=False)
  t_b = utils.tensor_from_array(sample_points_in_domain(0, t_max, Nb), device=device, requires_grad=False)
  xyzt_b = utils.stack_xyzt_tensors(x_b, y_b, z_b, t_b)

  # points & normal vectors on the surface of the wing
  ## sample Nw wing points with the corresponding normals
  sampled_indices = wing_df.sample(n=Nw).index

  x_w, y_w, z_w = [utils.tensor_from_array(wing_df.loc[sampled_indices, col].values, device=device, requires_grad=False) for col in ['x', 'y', 'z']]
  n_x, n_y, n_z = [utils.tensor_from_array(norm_df.loc[sampled_indices, col].values, device=device, requires_grad=False) for col in ['x', 'y', 'z']]
  t_w = utils.tensor_from_array(sample_points_in_domain(0, t_max, Nw), device=device, requires_grad=False)

  xyzt_w = utils.stack_xyzt_tensors(x_w, y_w, z_w, t_w)
  n_xyz = utils.stack_xyz_tensors(n_x, n_y, n_z)

  return (x_f, y_f, z_f, t_f, xyzt_0, xyzt_b, xyzt_w, n_xyz)

In [10]:
out_filepath = "/Users/ggito/repos/pinns/data/"
# torch.save(pinn, out_filepath + 'pinn10.pt')

In [11]:
def closure(report_losses=False):
  optimizer.zero_grad()

  training_input = create_training_inputs(x_max, y_max, z_max, t_max, Nf, N0, Nb, Nw)

  total_loss, pde_loss, ic_loss, bc_loss, no_slip_loss, imp_loss = pinn.loss(
                  *training_input,
                  in_velocity,
                  mu, rho,
                  1, 1, 1, 1, 1,
                  log_filepath)

  total_loss.backward()

  if report_losses:
      return total_loss, pde_loss, ic_loss, bc_loss, no_slip_loss, imp_loss
  else:
      return total_loss

for epoch in range(epochs):
  
  optimizer.step(closure)

  total_loss, pde_loss, ic_loss, bc_loss, no_slip_loss, imp_loss = closure(report_losses=True)
  print(f'Epoch: {epoch},\tTotal loss: {total_loss.item()},\tPDE loss: {pde_loss.item()},\tIC loss: {ic_loss.item()},\tBC loss: {bc_loss.item()},\tNo-slip loss: {no_slip_loss.item()},\tImpermeability loss: {imp_loss.item()}')

  if np.isnan(total_loss.item()):
    break

  if epoch % 100 == 0:
    torch.save(pinn, out_filepath + 'pinn11.pt')

Epoch: 0,	Total loss: 68.02820587158203,	PDE loss: 1.9007532596588135,	IC loss: 6.383991241455078,	BC loss: 39.526371002197266,	No-slip loss: 20.21708869934082,	Impermeability loss: 0
Epoch: 1,	Total loss: 60.642738342285156,	PDE loss: 0.4980859160423279,	IC loss: 7.354363918304443,	BC loss: 38.69352340698242,	No-slip loss: 14.096762657165527,	Impermeability loss: 0
Epoch: 2,	Total loss: 58.6119384765625,	PDE loss: 0.04701992869377136,	IC loss: 10.798709869384766,	BC loss: 37.22823715209961,	No-slip loss: 10.537971496582031,	Impermeability loss: 0
Epoch: 3,	Total loss: 57.92830276489258,	PDE loss: 0.18190434575080872,	IC loss: 10.03573989868164,	BC loss: 38.04023361206055,	No-slip loss: 9.670424461364746,	Impermeability loss: 0
Epoch: 4,	Total loss: 57.06977081298828,	PDE loss: 0.19689522683620453,	IC loss: 9.1410493850708,	BC loss: 38.17107391357422,	No-slip loss: 9.560751914978027,	Impermeability loss: 0
Epoch: 5,	Total loss: 56.539791107177734,	PDE loss: 0.392861008644104,	IC loss: 

In [ ]:
# out_filepath = "/Users/ggito/repos/pinns/data/"
# torch.save(pinn, out_filepath + 'pinn10.pt')

In [ ]:
# out_filepath = "/Users/ggito/repos/pinns/data/"
# pinn = torch.load(out_filepath + 'pinn8.pt')
# pinn.train()
# # loaded_model.eval()